In [1]:
# %%
from ChromFormer.generator import generate_biological_structure
from ChromFormer import plotting as pl

import os
from pathlib import Path

from sklearn.model_selection import ParameterGrid
import pandas as pd

from dotenv import load_dotenv
loaded = load_dotenv()

path_structures = Path('~/Downloads/ChromFormer-paper/create-structures-output/').expanduser()
path_structures.mkdir(parents=True, exist_ok=True)

In [2]:
NB_BINS = 202  # number of points per structure
DELTA = 0.45  # Smoothness parameter
ST_SIG = 5
END_SIG = 7
SIG = 4  # structure compactness
CLUST_SIG = 1.5  # TADs compactness
CLUST_PROB = 0.1  # Probability of entering a TAD
SECONDSTEP = False

SEED = 42
EXPONENT = 1  # root power value for the inverse function (Distance -> Hi-C)
ICING = True  # Whether to use ICE normalisation with Z_score or not
MINMAXUSE = False  # Whether MinMax needs to be used before optimal transport on the synthetic data or not
TRANSPORTATION = True  # Whether to use optimal transport or not
SOFTMAXING = False  # Whether to use a synthetic to true HiC softmax function or not. Not needed if already using optimal transport


In [3]:
scan = dict(nb_nodes=[NB_BINS],
                  delta=[0.275, 0.45, 0.9],
                  start_sigma=[2, 5, 10],
                  end_sigma=[3, 7, 14],
                  sigma=[2,4,8],
                  cluster_sigma=[0.75, 1.5, 3],
                  cluster_proba=[0.05, 0.1, 0.2],
                  step2=[True, False],
                  aging_step = [15, 30, 60]
                 )

defaults= dict(nb_nodes=202,
                  delta=0.45,
                  start_sigma=5,
                  end_sigma=7,
                  sigma=4,
                  cluster_sigma=1.5,
                  cluster_proba=0.1,
                  step2=False,
                  aging_step = 30)

In [4]:
param_grid = {key: [value] for key,value in defaults.items()}

key1, key2 = 'delta', 'start_sigma'
param_grid[key1] = scan[key1]
param_grid[key2] = scan[key2]
param_grid = ParameterGrid(param_grid)

In [5]:
df = pd.DataFrame.from_dict(param_grid)
df = df[[key1, key2]]

In [6]:
structures = [generate_biological_structure(**kwargs) for kwargs in param_grid]

In [7]:
fig = pl.structures_in_sphere(structures, param_grid_df=df)

In [8]:
import plotly.io as pio
pio.renderers.default = "browser"
fig.show()